# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
from pprint import pprint

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.21,97,100,23.75,TF,1738524757
1,1,albany,42.6001,-73.9662,-6.20,68,100,1.34,US,1738524277
2,2,grytviken,-54.2811,-36.5092,8.14,82,86,5.35,GS,1738524761
3,3,batouri,4.4333,14.3667,25.06,25,84,0.88,CM,1738524762
4,4,colonia,50.9333,6.9500,2.20,72,0,2.57,DE,1738524590


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lon",
    "Lat", 
    geo = True,
    tiles = "OSM",
    frame_width = 700, 
    frame_height = 500, 
    size = "Humidity",     #size of the point should be the humidity in each city
    color = "City", 
    scale = 0.8
    )

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values

# I do not enjoy temperatures that are too hot or too cold, but I do live in Minneapolis, so some warmth and sunshine are desired. 
# I am going to use a range of 24 to 32 for temperature and some cloud coverage 40 to 60.
ideal_weather_condition = city_data_df[(city_data_df["Max Temp"] >= 24) & (city_data_df["Max Temp"] <= 32) &
                                       (city_data_df["Cloudiness"] >= 35) & (city_data_df["Cloudiness"] <= 65)]

# Drop any rows with null values
ideal_weather_condition = ideal_weather_condition.dropna()

# Display sample data
ideal_weather_condition

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,tarawa,1.4190,172.9840,26.05,94,40,5.14,KI,1738524801
54,54,tondon,10.3667,-13.3500,25.26,22,57,0.65,GN,1738524824
65,65,panacan,9.2463,118.4082,25.55,82,62,2.72,PH,1738524837
119,119,abancay,-13.6339,-72.8814,24.51,45,57,3.70,PE,1738524901
147,147,broken hill,-31.9500,141.4333,27.44,41,55,5.09,AU,1738524934
166,166,tweed heads,-28.1817,153.5470,25.02,78,40,4.63,AU,1738524957
169,169,yokadouma,3.5167,15.0500,24.38,27,52,0.09,CM,1738524960
180,180,mejit,10.2753,170.8646,26.28,78,39,10.13,MH,1738524974
191,191,hithadhoo,-0.6000,73.0833,27.53,75,43,2.48,MV,1738524987
244,244,juan griego,11.0817,-63.9655,28.06,62,41,6.65,VE,1738525050


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_condition[["City", "Country", "Lat", "Lon", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lon,Humidity,Hotel Name
36,tarawa,KI,1.4190,172.9840,94,
54,tondon,GN,10.3667,-13.3500,22,
65,panacan,PH,9.2463,118.4082,82,
119,abancay,PE,-13.6339,-72.8814,45,
147,broken hill,AU,-31.9500,141.4333,41,
166,tweed heads,AU,-28.1817,153.5470,78,
169,yokadouma,CM,3.5167,15.0500,27,
180,mejit,MH,10.2753,170.8646,78,
191,hithadhoo,MV,-0.6000,73.0833,75,
244,juan griego,VE,11.0817,-63.9655,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Define the API parameters
radius = 10000 #meters
params = {
    "apiKey":geoapify_key,
    "format":"json", 
    "limit":1, 
    "categories":"accommodation.hotel"  #only search for hotels
}

# Set the base URL
base_url = "https://api.geoapify.com/v2/places"  # my endpoint will be a place (specifically a hotel)

In [36]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = row["Lon"]  #access the lon of the current row
    latitude = row["Lat"]   #access the lat of the current row
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Make the API request and convert response to JSON
    name_address = requests.get(base_url, params = params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tarawa - nearest hotel: Betio Lodge
tondon - nearest hotel: No hotel found
panacan - nearest hotel: Gorayan Lodge
abancay - nearest hotel: Hotel Turistas
broken hill - nearest hotel: Ibis Styles
tweed heads - nearest hotel: Dolphins Harbourside Hotel
yokadouma - nearest hotel: Hôtel Grand Moulin
mejit - nearest hotel: No hotel found
hithadhoo - nearest hotel: Pebbles Inn
juan griego - nearest hotel: Hotel Descanso Tropical
vallegrande - nearest hotel: Hotel Santa Cruz
kingscliff - nearest hotel: Kingscliff Beach Hotel
bikenibeu village - nearest hotel: Otintaai Hotel
kavaratti - nearest hotel: No hotel found
bousso - nearest hotel: No hotel found
paita - nearest hotel: Victor's
tolanaro - nearest hotel: Hôtel Mahavokey
bolobo - nearest hotel: No hotel found
manadhoo - nearest hotel: Soneva Jani
tafuna - nearest hotel: Tradewinds
pitimbu - nearest hotel: Pousada Brisa Mar
kourou - nearest hotel: SCI Horizon
freetown - nearest hotel: Formal Leona hotel
birao - neare

,City,Country,Lat,Lon,Humidity,Hotel Name
36,tarawa,KI,1.4190,172.9840,94,Betio Lodge
54,tondon,GN,10.3667,-13.3500,22,No hotel found
65,panacan,PH,9.2463,118.4082,82,Gorayan Lodge
119,abancay,PE,-13.6339,-72.8814,45,Hotel Turistas
147,broken hill,AU,-31.9500,141.4333,41,Ibis Styles
166,tweed heads,AU,-28.1817,153.5470,78,Dolphins Harbourside Hotel
169,yokadouma,CM,3.5167,15.0500,27,Hôtel Grand Moulin
180,mejit,MH,10.2753,170.8646,78,No hotel found
191,hithadhoo,MV,-0.6000,73.0833,75,Pebbles Inn
244,juan griego,VE,11.0817,-63.9655,62,Hotel Descanso Tropical


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lon",
    "Lat", 
    geo = True,
    tiles = "OSM",
    frame_width = 700, 
    frame_height = 500, 
    size = "Humidity",     #size of the point should be the humidity in each city
    color = "City", 
    scale = 0.8, 
    hover_cols=["City", "Hotel Name", "Country"]  # Columns to display in hover message
    )

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Humidity,Hotel Name,Country)